# Import all required packages

In [ ]:
!pip install librosa
!pip install pandas
!pip install transformers
!pip install jiwer
!pip install scikit-learn
!pip install torch
!pip install datasets
!pip install dataclasses
!pip install typing
!pip install numpy

In [11]:
import os
import re
from tqdm import tqdm
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
import pandas as pd
from jiwer import wer, cer
from sklearn.model_selection import train_test_split
from datasets import Dataset, ClassLabel, load_metric
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import random
import json
import numpy as np
import warnings

# Processing source files

Get all files, define new sort function to sort as \[1, 2, 3 ... 100\], because built-in function sorts strings as \[1, 100, 101 ...\].

In [15]:
def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

In [16]:
def prepare_files(directory, file_with_text, inf):
    with open(file_with_text, encoding='utf-16') as f:
        text = f.readlines()
    files = os.listdir(directory)
    files_full = []
    for filename in files:
        if '.DS_Store' not in filename:
            f = os.path.join(directory, filename)
            files_full.append(f)
    files_full.sort(key=natural_keys)
    j = 0
    dict_for_inf = []
    for filename in tqdm(files_full):
        if not '=' in text[j] and not 'нрзб' in text[j] and not '[' in text[j] and not '<' in text[j]:
            x = text[j].replace('\n', '').lower()
            x = x.replace('.', ' ')
            x = x.replace(',', ' ')
            x = x.replace(':', ' ')
            x = x.replace('?', ' ')
            x = x.replace('!', ' ')
            x = x.replace('–', ' ')
            x = x.replace('-', ' ')
            x = x.replace('ё', 'е')
            x = re.sub('(\s){2,}', ' ', x)
            x = re.sub('\(.*\)', '', x)
            x = x.rstrip()
            dict_for_inf.append({'respondent':inf, 'path': filename, 'sentence': x})
        j += 1  
    return dict_for_inf

In [17]:
enm = prepare_files('/content/input_opochka/new_mono_enm20180618', 
                    '/content/input_opochka/20180618_enm1930_1to487.txt', 'ENM1930')
ive = prepare_files('/content/input_opochka/new_mono_ive20190702', 
                    '/content/input_opochka/20190702_ive1949_1to234.txt', 'IVE1949')
onv = prepare_files('/content/input_opochka/new_mono_onv20180622', 
                    '/content/input_opochka/20180622_onv1972_1to529.txt', 'ONV1972')
saf = prepare_files('/content/input_opochka/new_mono_saf20190701', 
                    '/content/input_opochka/20190701_saf1973_1to434.txt', 'IVE1949')
tai = prepare_files('/content/input_opochka/new_mono_tai20190706', 
                    '/content/input_opochka/20190706_tai1955_1to167.txt', 'TAI1955')
tve = prepare_files('/content/input_opochka/new_mono_tve20190702', 
                    '/content/input_opochka/20190702_tve1955_1to709.txt', 'TVE1955')
vav = prepare_files('/content/input_opochka/new_mono_vav20180619', 
                    '/content/input_opochka/20180619_vav1949_1to277.txt', 'VAV1949')

100%|████████████████████████████████████████████████████████████████████████████| 434/434 [00:00<00:00, 216886.45it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████| 277/277 [00:00<00:00, 276822.07it/s]


In [18]:
all_data = enm + ive + onv + saf + tai + tve + vav
len(all_data)

2256

# Read audio

In [20]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["sentence"]
    return batch

test_dataset = []
for l in tqdm(all_data):
    test_dataset.append(speech_file_to_array_fn(l))
data = [d['speech'] for d in test_dataset]

100%|██████████████████████████████████████████████████████████████████████████████| 2256/2256 [01:49<00:00, 20.66it/s]


# Fine-tune

In [23]:
df = pd.DataFrame(test_dataset, columns=['respondent', 'path', 'sentence', 'speech'])
ds = Dataset.from_pandas(df[['sentence', 'speech']])
ds = ds.train_test_split(test_size=0.3, seed=22)

In [24]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=ds.column_names["train"])


Map:   0%|          | 0/1579 [00:00<?, ? examples/s]

Map:   0%|          | 0/677 [00:00<?, ? examples/s]

In [25]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

In [26]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [27]:
LANG_ID = "ru"
MODEL_ID = "bond005/wav2vec2-large-ru-golos-with-lm"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID, padding=True)
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [28]:
def prepare_dataset(batch, processor):
    audio = batch["speech"]
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [29]:
ds = ds.map(lambda examples: prepare_dataset(examples, processor))

Map:   0%|          | 0/1579 [00:00<?, ? examples/s]

C:\Users\Katya\anaconda3\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/677 [00:00<?, ? examples/s]

In [30]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",).to('cpu')
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",).to('cpu')

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100).to('cpu')
        batch["labels"] = labels.to('cpu')
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [32]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer", revision="master")

C:\Users\Katya\AppData\Local\Temp\ipykernel_1420\1788232804.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [33]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
    pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
    label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]
    
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer, 'cer': cer}

In [34]:
model = Wav2Vec2ForCTC.from_pretrained('/content/wav2vec2-large-ru-golos-with-lm-dialect-full/checkpoint-10410/', local_files_only=True)
model.freeze_feature_extractor()

C:\Users\Katya\anaconda3\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1644: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./wav2vec2-large-ru-golos-with-lm-shetnevo_and_opochka',
  per_device_train_batch_size=8,
  learning_rate=1e-4,
  evaluation_strategy="epoch",
  num_train_epochs=15,
  report_to="none",
  save_strategy="epoch"
)


In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=processor.feature_extractor,
)

In [37]:
device = torch.device("cpu")

In [38]:
warnings.filterwarnings('ignore')

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,Wer,Cer
1,No log,2.702313,0.531165,0.249459
2,No log,2.616474,0.521760,0.241440
3,1.212700,2.752938,0.509166,0.235312
4,1.212700,3.163397,0.503587,0.233300
5,1.212700,3.243788,0.493384,0.226074
6,0.878800,3.531349,0.493384,0.227873
7,0.878800,3.699901,0.497848,0.226897
8,0.705300,3.605258,0.490355,0.222934
9,0.705300,3.500510,0.488124,0.224214
10,0.705300,3.982125,0.487327,0.222720


TrainOutput(global_step=2970, training_loss=0.7405658606326941, metrics={'train_runtime': 124221.3891, 'train_samples_per_second': 0.191, 'train_steps_per_second': 0.024, 'total_flos': 6.411286297934812e+18, 'train_loss': 0.7405658606326941, 'epoch': 15.0})

# Testing

In [52]:
MODEL_ID = "bond005/wav2vec2-large-ru-golos-with-lm"
model = Wav2Vec2ForCTC.from_pretrained('/content/wav2vec2-large-ru-golos-with-lm-shetnevo_and_opochka/checkpoint-2970/', local_files_only=True)
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID, padding=True)

In [53]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

results = ds["test"].map(map_to_result, remove_columns=ds["test"].column_names)

Map:   0%|          | 0/677 [00:00<?, ? examples/s]

In [54]:
wers = []
cers = []


for item in results:
    if item['text'] != '' and item['text'] != ' ':
        w = wer(item['text'], item['pred_str'])
        wers.append(w)
        c = cer(item['text'], item['pred_str'])
        cers.append(c)

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.5366484431689765
Mean CER:  0.3036211593533453


In [58]:
test_results = results.to_pandas()
path = "/content/wav2vec_shetnevo_opochka.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

test_results.to_excel(writer) 

writer.save()
writer.close()

# Use a spellchecker for the received transcriptions

In [59]:
from pyaspeller import YandexSpeller
speller = YandexSpeller()
transcrtiptions_spelled = []
for t in tqdm(results['pred_str']):
    transcrtiptions_spelled.append(speller.spelled(t))

100%|████████████████████████████████████████████████████████████████████████████████| 677/677 [02:11<00:00,  5.13it/s]


In [61]:
wers = []
cers = []

for i, transcrtiption_spelled in enumerate(transcrtiptions_spelled):
    if results['text'][i] != '' and results['text'][i] != ' ':
        w = wer(results['text'][i], transcrtiption_spelled)
        wers.append(w)
        c = cer(results['text'][i], transcrtiption_spelled)
        cers.append(c)
        results['pred_str'][i] = transcrtiption_spelled

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.49522013704937784
Mean CER:  0.30350973125330377


In [62]:
test_results = results.to_pandas()
path = "/content/wav2vec_shetnevo_opochka_with_spellcheck.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

test_results.to_excel(writer) 

writer.save()
writer.close()